In [35]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import yaml
import os
import glob

In [36]:
CONFIG_PATH = "/home/office5/Image_Code/Ketan_sir/"
def load_config(config_name):
    with open(os.path.join(CONFIG_PATH, config_name)) as file:
        config = yaml.safe_load(file)
    return config

config = load_config("config.yaml")
config

{'t_lower': 100,
 't_upper': 200,
 'L2Gradient': True,
 'connectivity': 4,
 'measure_pixel': 4,
 'get_pixel_img1': 5,
 'get_pixel_img2': 10,
 'top_left_point_kernel': 1,
 'bottom_right_point_kernel': 2}

In [37]:
CONFIG_PATH = "/home/office5/Image_Code/Ketan_sir/config.yaml"
def load_config(CONFIG_PATH):
    with open(CONFIG_PATH) as file:
        config = yaml.safe_load(file)
    return config

config = load_config(CONFIG_PATH)
config

{'t_lower': 100,
 't_upper': 200,
 'L2Gradient': True,
 'connectivity': 4,
 'measure_pixel': 4,
 'get_pixel_img1': 5,
 'get_pixel_img2': 10,
 'top_left_point_kernel': 1,
 'bottom_right_point_kernel': 2}

In [38]:
def get_edge_image(img_path):
    # Assume the image come is cropped part
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray,45,255,cv2.THRESH_BINARY)[1]
    edge = cv2.Canny(thresh, config['t_lower'], config['t_upper'], L2gradient = config['L2Gradient'])
    return edge,gray

def get_connected_comp_img(edge):
    lst_component = []
    output = cv2.connectedComponentsWithStats(edge, config['connectivity'], cv2.CV_32S)
    (numLabels, labels, stats, centroids) = output

    # loop over the number of unique connected component labels
    for i in range(0, numLabels):
        if i == 0:
            text = "examining component {}/{} (background)".format(
                i + 1, numLabels)
        # otherwise, we are examining an actual connected component
        else:
            text = "examining component {}/{}".format( i + 1, numLabels)
        # x_ = stats[i, cv2.CC_STAT_LEFT]
        # y_ = stats[i, cv2.CC_STAT_TOP]
        w_ = stats[i, cv2.CC_STAT_WIDTH]
        h_ = stats[i, cv2.CC_STAT_HEIGHT]
        area = stats[i, cv2.CC_STAT_AREA]
        (cX, cY) = centroids[i]

        keepWidth = w_ > 2
        keepHeight = h_ > 2
        keepArea = area > 50

        if all((keepWidth, keepHeight, keepArea)):
            componentMask = (labels == i).astype("uint8") * 255
            # print(componentMask)
            # cv2.imwrite(f"/home/office5/Image_Code/Ketan_sir/try1_fun_component/{i}.png",componentMask)
            color_img  = cv2.cvtColor(componentMask,cv2.COLOR_GRAY2BGR)
            lst_component.append(color_img)
    return lst_component

In [39]:
def get_extreme_point(all_component_image):

    # to store all extreme point
    extr_point = []


    # Here define range from 1 to n because 0 image is backgroumd
    for i in range(1,len(all_component_image)):
        gray = cv2.cvtColor(all_component_image[i],cv2.COLOR_BGR2GRAY)
        coords = np.column_stack(np.where(gray == 255))

        # print(coords)
        a = np.min(coords,axis=0)
        b = np.max(coords,axis=0)

        y_min,x_min,y_max,x_max = a[0],a[1],b[0],b[1]

        points = []

        for i in range(len(coords)):
            if(x_min == coords[i][1]):
                points.append([coords[i][0], coords[i][1]]) 
                break
            else:
                pass
        points.append([coords[-1][0] , coords[-1][1]])
        points = np.array(points)

        for i in range(2):
            extr_point.append([points[i][0], points[i][1]])

    return extr_point
        


In [40]:
# def get_index_kernel(ori_gray,arr2,def_arr):

#     y1_s = arr2[0] + 2744 - config['top_left_point_kernel']
#     y2_s = arr2[0] + 2744 + config['bottom_right_point_kernel']
#     x1_s = arr2[1] + 1824 - config['top_left_point_kernel']
#     x2_s = arr2[1] + 1824 + config['bottom_right_point_kernel']
#     # print("gr",gray)
#     kernel_value =  ori_gray[y1_s:y2_s,x1_s:x2_s]
#     print("Kerenl_value", kernel_value)
#     count = 0
#     for i in range(len(kernel_value[0])):
#         for j in range(len(kernel_value[1])):
#             def_arr[i][j] = (np.subtract(int(kernel_value[config['top_left_point_kernel']][config['top_left_point_kernel']]),int(kernel_value[i][j])))
#             def_arr[config['top_left_point_kernel']][config['top_left_point_kernel']] = 5 
#             if(def_arr[i][j]==0):
#                 def_arr[i][j] = 1
#             elif(def_arr[i][j]<0):
#                 def_arr[i][j] = 0
#             else:
#                 def_arr[i][j] = 1
            
#     return def_arr

# trial only for part of image

In [41]:
def get_index_kernel(ori_gray,extreme_point_arr,set_kernel_value):

    y1_s = extreme_point_arr[0] - config['top_left_point_kernel']
    y2_s = extreme_point_arr[0] + config['bottom_right_point_kernel']
    x1_s = extreme_point_arr[1] - config['top_left_point_kernel']
    x2_s = extreme_point_arr[1] + config['bottom_right_point_kernel']
    try:
        kernel_value =  ori_gray[y1_s:y2_s,x1_s:x2_s]
        for i in range(len(kernel_value[0])):
            for j in range(len(kernel_value[1])):
                set_kernel_value[i][j] = (np.subtract(int(kernel_value[config['top_left_point_kernel']][config['top_left_point_kernel']]),int(kernel_value[i][j])))
                set_kernel_value[config['top_left_point_kernel']][config['top_left_point_kernel']] = 5 
                if(set_kernel_value[i][j]==0):
                    set_kernel_value[i][j] = 1
                elif(set_kernel_value[i][j]<0):
                    set_kernel_value[i][j] = 0
                else:
                    set_kernel_value[i][j] = 1
    except:
        # print(kernel_value)
        set_kernel_value = np.zeros((3,3))
        # print(set_kernel_value)
    return set_kernel_value

In [42]:
def get_removed_extreme_point(ori_gray,all_extreme_point):

    extreme_point_list = []
    dict1 = {}
    kernel_shape = config['top_left_point_kernel'] + config['bottom_right_point_kernel']
    m = 0
    dict1 = {}
    print("all point testing Point")
    for i in range(len(all_extreme_point)):
        extreme_point_arr = np.array(all_extreme_point[i])
        # print(arr)
        zero_kernel_arr = np.zeros((kernel_shape,kernel_shape))
        kernel_value = get_index_kernel(ori_gray,extreme_point_arr,zero_kernel_arr)
        unique_1_0 = np.unique(kernel_value,return_counts=True)
        # print(unique_1_0)
        if len(unique_1_0[0])==3:
            count_0 = unique_1_0[1][0]
            count_1 = unique_1_0[1][1]
            # print(count_0,count_1)
            dict1['0'] = count_0
            dict1['1'] = count_1
            if (count_0==3 and count_1==5) or (count_0==5 and count_1==3):

                corner_1 = kernel_value[0][0]
                corner_2 = kernel_value[0][2]
                corner_3 = kernel_value[2][0]
                corner_4 = kernel_value[2][2]

                corner_point = np.array([corner_1,corner_2,corner_3,corner_4])
                for key, value in dict1.items():
                    if value == 3:
                        corner_index = np.where(corner_point==int(key)) 

                        if corner_index[0][0] == 0:
                            x,y = 0,0
                            if (kernel_value[x][y]==int(key) and kernel_value[x+1][y]==int(key) and kernel_value[x][y+1]==int(key)):
                                # print('--------0',[a[i][0],a[i][1]])
                                extreme_point_list.append([a[i][0],a[i][1]])

                        if corner_index[0][0] == 1:
                            x,y = 0,2
                            if (kernel_value[x][y]==int(key) and kernel_value[x][y-1]==int(key) and kernel_value[x+1][y]==int(key)):
                                # print('--------1',[a[i][0],a[i][1]])
                                extreme_point_list.append([a[i][0],a[i][1]])

                        if corner_index[0][0] == 2:
                            x,y = 2,0
                            if (kernel_value[x][y]==int(key) and kernel_value[x-1][y]==int(key) and kernel_value[x][y+1]==int(key)):
                                # print('--------2',[a[i][0],a[i][1]])
                                extreme_point_list.append([a[i][0],a[i][1]])

                        if corner_index[0][0] == 3:
                            x,y = 2,2
                            if (kernel_value[x][y]==int(key) and kernel_value[x][y-1]==int(key) and kernel_value[x-1][y]==int(key)):
                                # print('--------3',[a[i][0],a[i][1]])
                                extreme_point_list.append([a[i][0],a[i][1]])


    return extreme_point_list

# a = np.load('coordinate.npy')
# a = [tuple(row) for row in a]
# a = np.unique(a,axis=0)
# h = config['top_left_point_kernel'] + config['bottom_right_point_kernel']

# def_arr = np.zeros((h,h))
# dict1 = {}
# extrmem_point_list = []
# value = get_extreme_point(a,dict1,extrmem_point_list)
# print(value)

In [44]:
img_path = '/home/office5/Image_Code/Ketan_sir/gray.png'
edge,original_gray = get_edge_image(img_path)
all_component_image = get_connected_comp_img(edge)
all_extreme_point =  get_extreme_point(all_component_image)
print("-"*80)
print(all_extreme_point)
remove_duplicate_extreme_point = [tuple(row) for row in all_extreme_point]
remove_duplicate_extreme_point = np.unique(remove_duplicate_extreme_point,axis=0)
# print("$"*30)
# print(a)
value = get_removed_extreme_point(original_gray,remove_duplicate_extreme_point)
print("*"*80)
print(value)
# For priting white point on extreme point pixel
for i in range(len(value)):
    original_gray[value[i][0]][value[i][1]] = 255
cv2.imwrite('e.png',original_gray)

--------------------------------------------------------------------------------
[[0, 6], [106, 42], [6, 63], [19, 66], [102, 16], [111, 39], [0, 162], [135, 208], [73, 189], [80, 210], [0, 268], [64, 274], [0, 313], [53, 317], [0, 367], [75, 376], [83, 495], [83, 495], [84, 497], [85, 497], [24, 524], [51, 524], [51, 526], [51, 526], [4, 28], [15, 31], [25, 440], [32, 456], [123, 21], [125, 46], [34, 391], [46, 415], [113, 453], [115, 456], [56, 395], [57, 415], [126, 446], [129, 462], [59, 288], [66, 309], [68, 246], [75, 268], [88, 248], [90, 270], [101, 352], [106, 370], [93, 199], [110, 206], [81, 346], [91, 371], [129, 161], [157, 233], [165, 188], [172, 234], [231, 728], [231, 735], [205, 596], [210, 647], [209, 199], [216, 248], [290, 750], [290, 763], [211, 442], [294, 453], [221, 470], [293, 480], [229, 642], [286, 669], [264, 186], [295, 197], [237, 218], [240, 240], [216, 526], [285, 581], [228, 587], [285, 630], [213, 701], [283, 761], [224, 301], [287, 355], [270, 362], [

True